<a href="https://colab.research.google.com/github/honzas83/t5s/blob/main/examples/t5s_aclimdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/honzas83/t5s --upgrade

  Cloning https://github.com/honzas83/t5s to /tmp/pip-req-build-1wv2q_z0
  Running command git clone -q https://github.com/honzas83/t5s /tmp/pip-req-build-1wv2q_z0
  Created wheel for t5s: filename=t5s-0.1-cp36-none-any.whl size=13589 sha256=d73939fb77acddb852531e952994a706002758fc339bd661a23a09156b0ff179
  Stored in directory: /tmp/pip-ephem-wheel-cache-7c19qz39/wheels/04/e5/71/24b59a9d225bfaead43ca97afe95fce46b5d56ddba98ac4b2d
Successfully built t5s
  Found existing installation: t5s 0.1
    Uninstalling t5s-0.1:
      Successfully uninstalled t5s-0.1


In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-12-10 12:54:45--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.2’

aclImdb_v1.tar.gz.2 100%[===================>]  80.23M  23.3MB/s    in 3.4s    

2020-12-10 12:54:48 (23.3 MB/s) - ‘aclImdb_v1.tar.gz.2’ saved [84125825/84125825]



In [3]:
!tar xzf aclImdb_v1.tar.gz

In [4]:
import os
from glob import glob
import random

In [5]:
def find_data(dn):
    fns = glob(os.path.join(dn, "pos", "*.txt"))+glob(os.path.join(dn, "neg", "*.txt"))
    return fns

def convert_data(fns, out_fn):
    random.shuffle(fns)
    with open(out_fn, "w", encoding="utf-8") as fw:
        for fn in fns:
            if "/pos/" in fn:
                label = "positive"
            elif "/neg/" in fn:
                label = "negative"
            else:
                continue
            with open(fn, "r", encoding="utf-8") as fr:
                text = fr.read().strip()
            if not text:
                continue
            
            text = text.replace("\n", " ").replace("\t", " ")
            print(text, label, sep="\t", file=fw)

In [6]:
train_fns = find_data("aclImdb/train")
dev_fns = train_fns[-2000:]
del train_fns[-2000:]
test_fns = find_data("aclImdb/test")
convert_data(train_fns, "aclImdb.train.tsv")
convert_data(dev_fns, "aclImdb.dev.tsv")
convert_data(test_fns, "aclImdb.test.tsv")

In [7]:
!wc -l *.tsv

    2000 aclImdb.dev.tsv
   25000 aclImdb.test.tsv
   23000 aclImdb.train.tsv
   50000 total


In [8]:
!gsutil cp -r gs://t5-data/vocabs/cc_all.32000/ .

Copying gs://t5-data/vocabs/cc_all.32000/sentencepiece.model...
Copying gs://t5-data/vocabs/cc_all.32000/sentencepiece.vocab...
/ [2 files][  1.3 MiB/  1.3 MiB]                                                
Operation completed over 2 objects/1.3 MiB.                                      


In [9]:
config = {
    "tokenizer": {
        "spm": "cc_all.32000/sentencepiece.model",
    },
    "t5_model": {
        "pre_trained": "t5-base",
        "save_checkpoint": "T5_aclImdb",
        "save_checkpoint_every": 1,
    },
    "dataset": {
        "train_tsv": "aclImdb.train.tsv",
        "devel_tsv": "aclImdb.dev.tsv",
        "test_tsv": "aclImdb.test.tsv",
        "loader": {
            "input_size": 3072,
            "output_size": 256,
            "min_batch_size": 4,
        },
    },
    "training": {
        "shared_trainable": False,
        "encoder_trainable": True,
        "n_epochs": 20,
        "initial_epoch": 0,
        "steps_per_epoch": 1000,
        "learning_rate": 0.001,
        "learning_rate_schedule": True,
    },
}

In [10]:
from t5s import T5

In [11]:
t5 = T5(config)

In [ ]:
t5.fine_tune()

All model checkpoint weights were used when initializing T5Training.

Some weights of T5Training were not initialized from the model checkpoint at t5-base and are newly initialized: ['loss']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/20
1000/1000 [==============================] - 934s 934ms/step - sent_accuracy: 0.8488 - edit_accuracy: 0.9239 - loss: 0.2186 - val_sent_accuracy: 0.8730 - val_edit_accuracy: 0.9365 - val_loss: 0.1774

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0007071067811865475.
Epoch 2/20
1000/1000 [==============================] - 935s 935ms/step - sent_accuracy: 0.8961 - edit_accuracy: 0.9480 - loss: 0.1383 - val_sent_accuracy: 0.8900 - val_edit_accuracy: 0.9450 - val_loss: 0.1281

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0005773502691896258.
Epoch 3/20
1000/1000 [==============================] - 934s 934ms/step - sent_accuracy: 0.9029 - edit_accuracy: 0.9515 - loss: 0.1261 - val_sent_accuracy: 0.9005 - val_edit_accuracy: 0.9503 - val_loss: 0.1403

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 4/20
 771/1000 [======================>.......] - ETA: 